In [4]:
### 🧪 Salsa20 Core — One Round with Intermediate Steps

import numpy as np

# Suppress harmless overflow warnings from uint32 arithmetic
np.seterr(over='ignore')

# 🔁 Rotate left (circular left shift) — 32-bit
def rotl32(x, n):
    return ((x << n) | (x >> (32 - n))) & 0xffffffff

# 🔍 Quarter-round: ARX operations
def quarterround(y0, y1, y2, y3):
    print(f"  ▸ Quarter input: {y0:08x} {y1:08x} {y2:08x} {y3:08x}")
    y1 ^= rotl32(np.uint32(np.uint32(y0) + np.uint32(y3)), 7)
    y2 ^= rotl32(np.uint32(np.uint32(y1) + np.uint32(y0)), 9)
    y3 ^= rotl32(np.uint32(np.uint32(y2) + np.uint32(y1)), 13)
    y0 ^= rotl32(np.uint32(np.uint32(y3) + np.uint32(y2)), 18)
    print(f"  ▸ Quarter output: {y0:08x} {y1:08x} {y2:08x} {y3:08x}\n")
    return y0, y1, y2, y3

# 🧱 Print state as a 4×4 grid
def print_state(state, title="State"):
    print(f"\n🔹 {title}:")
    M = state.reshape((4, 4))
    for row in M:
        print(" ".join(f"{x:08x}" for x in row))

# 📦 Initial Salsa20 state (512 bits = 16 words × 32 bits)
# Constants + 256-bit key + counter + 64-bit nonce
state = np.array([
    0x61707865, 0x3120646e, 0x79622d36, 0x6b206574,  # constants: "expand 32-byte k"
    0x03020100, 0x07060504, 0x0b0a0908, 0x0f0e0d0c,  # key[0..15]
    0x00000001, 0x00000000,                         # counter
    0x1b1a1918, 0x1f1e1d1c,                         # nonce
    0x13121110, 0x17161514, 0x1b1a1918, 0x1f1e1d1c  # key[16..31]
], dtype=np.uint32)

print_state(state, "Initial State")

# 🧱 Column round (modifies columns 0..3)
def column_round(s):
    print("\n🔄 COLUMN ROUND")
    s[0], s[4], s[8], s[12] = quarterround(s[0], s[4], s[8], s[12])
    s[1], s[5], s[9], s[13] = quarterround(s[1], s[5], s[9], s[13])
    s[2], s[6], s[10], s[14] = quarterround(s[2], s[6], s[10], s[14])
    s[3], s[7], s[11], s[15] = quarterround(s[3], s[7], s[11], s[15])
    return s

# 🧱 Row round (modifies rows)
def row_round(s):
    print("\n🔁 ROW ROUND")
    s[0], s[1], s[2], s[3] = quarterround(s[0], s[1], s[2], s[3])
    s[5], s[6], s[7], s[4] = quarterround(s[5], s[6], s[7], s[4])
    s[10], s[11], s[8], s[9] = quarterround(s[10], s[11], s[8], s[9])
    s[15], s[12], s[13], s[14] = quarterround(s[15], s[12], s[13], s[14])
    return s

# 🌀 Apply 1 round = column + row round, with logs
working_state = state.copy()
working_state = column_round(working_state)
print_state(working_state, "After Column Round")

working_state = row_round(working_state)
print_state(working_state, "After Row Round (i.e., 1 Salsa20 round complete)")



🔹 Initial State:
61707865 3120646e 79622d36 6b206574
03020100 07060504 0b0a0908 0f0e0d0c
00000001 00000000 1b1a1918 1f1e1d1c
13121110 17161514 1b1a1918 1f1e1d1c

🔄 COLUMN ROUND
  ▸ Quarter input: 61707865 03020100 00000001 13121110
  ▸ Quarter output: 785c930c 4246bbba 6e683f46 cc720705

  ▸ Quarter input: 3120646e 07060504 00000000 17161514
  ▸ Quarter output: 9e5ce127 1c3ac420 b6511c9a 6b014f45

  ▸ Quarter input: 79622d36 0b0a0908 1b1a1918 1b1a1918
  ▸ Quarter output: 1f7db0e9 35292e42 0dace845 d9caf142

  ▸ Quarter input: 6b206574 0f0e0d0c 1f1e1d1c 1f1e1d1c
  ▸ Quarter output: 50c4497b 104f4549 c04b67ea 4ab8670f


🔹 After Column Round:
785c930c 9e5ce127 1f7db0e9 50c4497b
4246bbba 1c3ac420 35292e42 104f4549
6e683f46 b6511c9a 0dace845 c04b67ea
cc720705 6b014f45 d9caf142 4ab8670f

🔁 ROW ROUND
  ▸ Quarter input: 785c930c 9e5ce127 1f7db0e9 50c4497b
  ▸ Quarter output: 1985bd97 0e32a2c3 01162fe4 4a90a892

  ▸ Quarter input: 1c3ac420 35292e42 104f4549 4246bbba
  ▸ Quarter output: e813a2d